In [16]:
# 加载相关库
import paddle
import paddle.fluid as fluid
import paddle.fluid.dygraph as dygraph
from paddle.fluid.dygraph import Linear
import numpy as np
import random

* 1.paddle/fluid：飞桨的主库，目前大部分的实用函数均在paddle.fluid包内。
* 2.dygraph：动态图的类库。
* 3.Linear：神经网络的全连接层函数，即包含所有输入权重相加和激活函数的基本神经元结构。在房价预测任务中，使用只有一层的神经网络（全连接层）来实现线性回归模型

## 数据处理

In [17]:
def load_data():
    # 从文件导入数据
    datafile = 'housing.data'
    data = np.fromfile(datafile, sep=' ')

    # 每条数据包括14项，其中前面13项是影响因素，第14项是相应的房屋价格中位数
    feature_names = [ 'CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', \
                      'DIS', 'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT', 'MEDV' ]
    feature_num = len(feature_names)

    # 将原始数据进行Reshape，变成[N, 14]这样的形状
    data = data.reshape([data.shape[0] // feature_num, feature_num])

    # 将原数据集拆分成训练集和测试集
    # 这里使用80%的数据做训练，20%的数据做测试
    # 测试集和训练集必须是没有交集的
    ratio = 0.8
    offset = int(data.shape[0] * ratio)
    training_data = data[:offset]

    # 计算train数据集的最大值，最小值，平均值
    maximums, minimums, avgs = training_data.max(axis=0), training_data.min(axis=0), \
                                 training_data.sum(axis=0) / training_data.shape[0]
    
    # 记录数据的归一化参数，在预测时对数据做归一化
    global max_values
    global min_values
    global avg_values
    max_values = maximums
    min_values = minimums
    avg_values = avgs

    # 对数据进行归一化处理
    for i in range(feature_num):
        #print(maximums[i], minimums[i], avgs[i])
        data[:, i] = (data[:, i] - avgs[i]) / (maximums[i] - minimums[i])

    # 训练集和测试集的划分比例
    #ratio = 0.8
    #offset = int(data.shape[0] * ratio)
    training_data = data[:offset]
    test_data = data[offset:]
    return training_data, test_data

## 模型设计
### 实现过程分如下两步：

* 1.定义init函数：在类的初始化函数中声明每一层网络的实现函数。在房价预测模型中，只需要定义一层全连接层，模型结构和《使用Python和Numpy构建神经网络模型》章节模型保持一致。
* 2.定义forward函数：构建神经网络结构，实现前向计算过程，并返回预测结果，在本任务中返回的是房价预测结果。

In [18]:
class Regressor(fluid.dygraph.Layer):
    def __init__(self):
        super(Regressor, self).__init__()
        
        # 定义一层全连接层，输出维度是1，激活函数为None，即不使用激活函数
        self.fc = Linear(input_dim=13, output_dim=1, act=None)
    
    # 网络的前向计算函数
    def forward(self, inputs):
        x = self.fc(inputs)
        return x

## 训练配置
* 1.以guard函数指定运行训练的机器资源，表明在with作用域下的程序均执行在本机的CPU资源上。dygraph.guard表示在with作用域下的程序会以飞桨动态图的模式执行（实时执行）。
* 2.声明定义好的回归模型Regressor实例，并将模型的状态设置为训练。
* 3.使用load_data函数加载训练数据和测试数据。
* 4.设置优化算法和学习率，优化算法采用随机梯度下降SGD，学习率设置为0.01。

In [19]:
# 定义飞桨动态图的工作环境
with fluid.dygraph.guard():
    # 声明定义好的线性回归模型
    model = Regressor()
    # 开启模型训练模式
    model.train()
    # 加载数据
    training_data, test_data = load_data()
    # 定义优化算法，这里使用随机梯度下降-SGD
    # 学习率设置为0.01
    opt = fluid.optimizer.SGD(learning_rate=0.01, parameter_list=model.parameters())

## 训练过程
* 训练过程采用二层循环嵌套方式：

* 内层循环： 负责整个数据集的一次遍历，采用分批次方式（batch）。假设数据集样本数量为1000，一个批次有10个样本，则遍历一次数据集的批次数量是1000/10=100，即内层循环需要执行100次。

 for iter_id, mini_batch in enumerate(mini_batches):

* 外层循环： 定义遍历数据集的次数，通过参数EPOCH_NUM设置。

for epoch_id in range(EPOCH_NUM):

* 说明:
* batch的取值会影响模型训练效果。batch过大，会增大内存消耗和计算时间，且效果并不会明显提升；batch过小，每个batch的样本数据将没有统计意义。由于房价预测模型的训练数据集较小，我们将batch为设置10。
* 内层循环需要经过的步骤
* 1.数据准备：将一个批次的数据转变成np.array和内置格式。
* 2.前向计算：将一个批次的样本数据灌入网络中，计算输出结果。
* 3.计算损失函数：以前向计算结果和真实房价作为输入，通过损失函数square_error_cost计算出损失函数值（Loss）。飞桨所有的API接口都有完整的说明和使用案例，在后续的资深教程中我们会详细介绍API的查阅方法。
* 4.反向传播：执行梯度反向传播backward函数，即从后到前逐层计算每一层的梯度，并根据设置的优化算法更新参数opt.minimize。

In [20]:
with dygraph.guard(fluid.CPUPlace()):
    EPOCH_NUM = 10   # 设置外层循环次数
    BATCH_SIZE = 10  # 设置batch大小
    
    # 定义外层循环
    for epoch_id in range(EPOCH_NUM):
        # 在每轮迭代开始之前，将训练数据的顺序随机的打乱
        np.random.shuffle(training_data)
        # 将训练数据进行拆分，每个batch包含10条数据
        mini_batches = [training_data[k:k+BATCH_SIZE] for k in range(0, len(training_data), BATCH_SIZE)]
        # 定义内层循环
        for iter_id, mini_batch in enumerate(mini_batches):
            x = np.array(mini_batch[:, :-1]).astype('float32') # 获得当前批次训练数据
            y = np.array(mini_batch[:, -1:]).astype('float32') # 获得当前批次训练标签（真实房价）
            # 将numpy数据转为飞桨动态图variable形式
            house_features = dygraph.to_variable(x)
            prices = dygraph.to_variable(y)
            
            # 前向计算
            predicts = model(house_features)
            
            # 计算损失
            loss = fluid.layers.square_error_cost(predicts, label=prices)
            avg_loss = fluid.layers.mean(loss)
            if iter_id%20==0:
                print("epoch: {}, iter: {}, loss is: {}".format(epoch_id, iter_id, avg_loss.numpy()))
            
            # 反向传播
            avg_loss.backward()
            # 最小化loss,更新参数
            opt.minimize(avg_loss)
            # 清除梯度
            model.clear_gradients()
    # 保存模型
    fluid.save_dygraph(model.state_dict(), 'LR_model')

epoch: 0, iter: 0, loss is: [0.09431855]
epoch: 0, iter: 20, loss is: [0.16147684]
epoch: 0, iter: 40, loss is: [0.13630566]
epoch: 1, iter: 0, loss is: [0.04257344]
epoch: 1, iter: 20, loss is: [0.07768445]
epoch: 1, iter: 40, loss is: [0.13916549]
epoch: 2, iter: 0, loss is: [0.03336889]
epoch: 2, iter: 20, loss is: [0.09531576]
epoch: 2, iter: 40, loss is: [0.0148153]
epoch: 3, iter: 0, loss is: [0.10384295]
epoch: 3, iter: 20, loss is: [0.09832788]
epoch: 3, iter: 40, loss is: [0.15973365]
epoch: 4, iter: 0, loss is: [0.06052698]
epoch: 4, iter: 20, loss is: [0.0710202]
epoch: 4, iter: 40, loss is: [0.02641783]
epoch: 5, iter: 0, loss is: [0.05533341]
epoch: 5, iter: 20, loss is: [0.04728326]
epoch: 5, iter: 40, loss is: [0.16268376]
epoch: 6, iter: 0, loss is: [0.10269071]
epoch: 6, iter: 20, loss is: [0.03606953]
epoch: 6, iter: 40, loss is: [0.02419707]
epoch: 7, iter: 0, loss is: [0.02922444]
epoch: 7, iter: 20, loss is: [0.01943639]
epoch: 7, iter: 40, loss is: [0.04721181]
ep

In [21]:
# 定义飞桨动态图工作环境
with fluid.dygraph.guard():
    # 保存模型参数，文件名为LR_model
    fluid.save_dygraph(model.state_dict(), 'LR_model')
    print("模型保存成功，模型参数保存在LR_model中")

模型保存成功，模型参数保存在LR_model中


## 测试模型
### 下面我们选择一条数据样本，测试下模型的预测效果。测试过程和在应用场景中使用模型的过程一致，主要可分成如下三个步骤：
* 1.配置模型预测的机器资源。本案例默认使用本机，因此无需写代码指定。
* 2.将训练好的模型参数加载到模型实例中。由两个语句完成，第一句是从文件中读取模型参数；第二句是将参数内容加载到模型。加载完毕后，需要将模型的状态调整为eval()（校验）。上文中提到，训练状态的模型需要同时支持前向计算和反向传导梯度，模型的实现较为臃肿，而校验和预测状态的模型只需要支持前向计算，模型的实现更加简单，性能更好。
* 3.将待预测的样本特征输入到模型中，打印输出的预测结果。

In [22]:
def load_one_example(data_dir):
    f = open(data_dir, 'r')
    datas = f.readlines()
    # 选择倒数第10条数据用于测试
    tmp = datas[-10]
    tmp = tmp.strip().split()
    one_data = [float(v) for v in tmp]

    # 对数据进行归一化处理
    for i in range(len(one_data)-1):
        one_data[i] = (one_data[i] - avg_values[i]) / (max_values[i] - min_values[i])

    data = np.reshape(np.array(one_data[:-1]), [1, -1]).astype(np.float32)
    label = one_data[-1]
    return data, label

In [23]:
with dygraph.guard():
    # 参数为保存模型参数的文件地址
    model_dict, _ = fluid.load_dygraph('LR_model')
    model.load_dict(model_dict)
    model.eval()

    # 参数为数据集的文件地址
    test_data, label = load_one_example('housing.data')
    # 将数据转为动态图的variable格式
    test_data = dygraph.to_variable(test_data)
    results = model(test_data)

    # 对结果做反归一化处理
    results = results * (max_values[-1] - min_values[-1]) + avg_values[-1]
    print("Inference result is {}, the corresponding label is {}".format(results.numpy(), label))

Inference result is [[16.16264]], the corresponding label is 19.7
